In [16]:
import torch
from patsy.util import pandas_Categorical_categories
from streamlit import columns
from torchvision import models, transforms
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader
import pandas as pd

# Charger ResNet préentraîné
from torchvision.models import resnet50, ResNet50_Weights

weights = ResNet50_Weights.DEFAULT
resnet = resnet50(weights=weights)
resnet.eval()

# Si GPU Metal est disponible, basculer sur le GPU
device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
resnet = resnet.to(device)

# Transformation pour ResNet
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Charger CIFAR-10
dataset = CIFAR10(root="./data", train=True, transform=transform, download=False)
dataloader = DataLoader(dataset, batch_size=32, shuffle=False)

embeddings = []
labels = []

# Générer les embeddings
with torch.no_grad():
    for images, targets in dataloader:
        images = images.to(device)  # Envoie les images sur le GPU
        features = resnet(images)
        embeddings.append(features.cpu().numpy())  # Récupère les résultats sur le CPU
        labels.extend(targets.numpy())

# Sauvegarder dans un fichier CSV
embeddings_flat = [item for sublist in embeddings for item in sublist]  # Aplatir la liste
df = pd.DataFrame(embeddings_flat)
df['label'] = labels
df.to_csv("ML_deployement/data/ref_data.csv", index=False)

print("Embeddings générés et sauvegardés dans ML_deployement/data/ref_data.csv")

KeyboardInterrupt: 

In [17]:
pip install fastapi

  Obtaining dependency information for fastapi from https://files.pythonhosted.org/packages/54/c4/148d5046a96c428464557264877ae5a9338a83bbe0df045088749ec89820/fastapi-0.115.5-py3-none-any.whl.metadata
  Using cached fastapi-0.115.5-py3-none-any.whl.metadata (27 kB)
  Obtaining dependency information for starlette<0.42.0,>=0.40.0 from https://files.pythonhosted.org/packages/96/00/2b325970b3060c7cecebab6d295afe763365822b1306a12eeab198f74323/starlette-0.41.3-py3-none-any.whl.metadata
  Using cached starlette-0.41.3-py3-none-any.whl.metadata (6.0 kB)
  Obtaining dependency information for pydantic!=1.8,!=1.8.1,!=2.0.0,!=2.0.1,!=2.1.0,<3.0.0,>=1.7.4 from https://files.pythonhosted.org/packages/d5/74/da832196702d0c56eb86b75bfa346db9238617e29b0b7ee3b8b4eccfe654/pydantic-2.10.2-py3-none-any.whl.metadata
  Using cached pydantic-2.10.2-py3-none-any.whl.metadata (170 kB)
  Obtaining dependency information for annotated-types>=0.6.0 from https://files.pythonhosted.org/packages/78/b6/6307fbef88d9b5

In [18]:
pip install uvicorn

  Obtaining dependency information for uvicorn from https://files.pythonhosted.org/packages/50/c1/2d27b0a15826c2b71dcf6e2f5402181ef85acf439617bb2f1453125ce1f3/uvicorn-0.32.1-py3-none-any.whl.metadata
  Using cached uvicorn-0.32.1-py3-none-any.whl.metadata (6.6 kB)
  Obtaining dependency information for click>=7.0 from https://files.pythonhosted.org/packages/00/2e/d53fa4befbf2cfa713304affc7ca780ce4fc1fd8710527771b58311a3229/click-8.1.7-py3-none-any.whl.metadata
  Using cached click-8.1.7-py3-none-any.whl.metadata (3.0 kB)
Using cached uvicorn-0.32.1-py3-none-any.whl (63 kB)
Using cached click-8.1.7-py3-none-any.whl (97 kB)

[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [19]:
pip install torch torchvision pandas joblib pillow

  Obtaining dependency information for joblib from https://files.pythonhosted.org/packages/91/29/df4b9b42f2be0b623cbd5e2140cafcaa2bef0759a00b7b70104dcfe2fb51/joblib-1.4.2-py3-none-any.whl.metadata
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
Using cached joblib-1.4.2-py3-none-any.whl (301 kB)

[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [20]:
python -c "import fastapi; print('FastAPI installé avec succès!')"
python -c "import uvicorn; print('Uvicorn installé avec succès!')"

SyntaxError: invalid syntax (4109809331.py, line 1)

In [21]:
pip install scikit-learn

  Obtaining dependency information for scikit-learn from https://files.pythonhosted.org/packages/1b/be/386ef63d9d5e2ddf8308f6a164e4b388d5c5aecc0504d25acc6b33d8b09e/scikit_learn-1.5.2-cp39-cp39-macosx_12_0_arm64.whl.metadata
  Using cached scikit_learn-1.5.2-cp39-cp39-macosx_12_0_arm64.whl.metadata (13 kB)
  Obtaining dependency information for scipy>=1.6.0 from https://files.pythonhosted.org/packages/5c/c0/e71b94b20ccf9effb38d7147c0064c08c622309fd487b1b677771a97d18c/scipy-1.13.1-cp39-cp39-macosx_12_0_arm64.whl.metadata
  Using cached scipy-1.13.1-cp39-cp39-macosx_12_0_arm64.whl.metadata (60 kB)
  Obtaining dependency information for threadpoolctl>=3.1.0 from https://files.pythonhosted.org/packages/4b/2c/ffbf7a134b9ab11a67b0cf0726453cedd9c5043a4fe7a35d1cefa9a1bcfb/threadpoolctl-3.5.0-py3-none-any.whl.metadata
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
Using cached scikit_learn-1.5.2-cp39-cp39-macosx_12_0_arm64.whl (11.0 MB)
Using cached scipy-1.13.1-cp39-cp39-m

In [22]:
pip install python-multipart

  Obtaining dependency information for python-multipart from https://files.pythonhosted.org/packages/b4/fb/275137a799169392f1fa88fff2be92f16eee38e982720a8aaadefc4a36b2/python_multipart-0.0.17-py3-none-any.whl.metadata

[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [24]:
pip install -r /Users/maximemoreau/Documents/ML/ML_deployement/MLproj/serving/requierement.txt


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [25]:
pip install streamlit

  Obtaining dependency information for streamlit from https://files.pythonhosted.org/packages/ae/53/418536f5d0b87bfbe7bbd8c001983c27e9474f82723bd2e529660fd9a534/streamlit-1.40.2-py2.py3-none-any.whl.metadata
  Obtaining dependency information for altair<6,>=4.0 from https://files.pythonhosted.org/packages/aa/f3/0b6ced594e51cc95d8c1fc1640d3623770d01e4969d29c0bd09945fafefa/altair-5.5.0-py3-none-any.whl.metadata
  Obtaining dependency information for blinker<2,>=1.0.0 from https://files.pythonhosted.org/packages/10/cb/f2ad4230dc2eb1a74edf38f1a38b9b52277f75bef262d8908e60d957e13c/blinker-1.9.0-py3-none-any.whl.metadata
  Obtaining dependency information for cachetools<6,>=4.0 from https://files.pythonhosted.org/packages/a4/07/14f8ad37f2d12a5ce41206c21820d8cb6561b728e51fad4530dff0552a67/cachetools-5.5.0-py3-none-any.whl.metadata
  Using cached cachetools-5.5.0-py3-none-any.whl.metadata (5.3 kB)
  Obtaining dependency information for protobuf<6,>=3.20 from https://files.pythonhosted.org/packa

In [27]:
pip install -r MLproj/webapp/requirements.txt

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'MLproj/webapp/requirements.txt'

[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [28]:
pip install -r /Users/maximemoreau/Documents/ML/ML_deployement/MLproj/webapp/requierement.txt


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [29]:
pip install evidently

  Obtaining dependency information for evidently from https://files.pythonhosted.org/packages/a6/19/f37d8c0db248b3e5f8e2f7a1d238debbc1052c89c6a1821f14567bd26e08/evidently-0.4.40-py3-none-any.whl.metadata
  Obtaining dependency information for plotly>=5.10.0 from https://files.pythonhosted.org/packages/e5/ae/580600f441f6fc05218bd6c9d5794f4aef072a7d9093b291f1c50a9db8bc/plotly-5.24.1-py3-none-any.whl.metadata
  Obtaining dependency information for statsmodels>=0.12.2 from https://files.pythonhosted.org/packages/dc/02/df44d1a73368fd0c0618e3169e7649303e6adb3ce96a429b617549f87165/statsmodels-0.14.4-cp39-cp39-macosx_11_0_arm64.whl.metadata
  Obtaining dependency information for nltk>=3.6.7 from https://files.pythonhosted.org/packages/4d/66/7d9e26593edda06e8cb531874633f7c2372279c3b0f46235539fe546df8b/nltk-3.9.1-py3-none-any.whl.metadata
  Obtaining dependency information for litestar>=2.8.3 from https://files.pythonhosted.org/packages/43/1b/31c8d75c98f69748c49d7b4196539d0e56b3c785f2b6b8e6b4686

In [30]:
pip install pandas


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [31]:
import pandas as pd
df = pd.read_csv("/Users/maximemoreau/Documents/ML/ML_deployement/MLproj/data/ref_data.csv")

In [32]:
df

,0,1,2,3,4,5,6,7,8,9,...,991,992,993,994,995,996,997,998,999,label
0,-0.226760,0.069927,-0.549122,-0.880585,-0.396384,0.033055,-0.109720,-0.748466,0.034720,-0.334325,...,0.025356,-0.042504,1.020076,0.187329,1.068738,1.399755,0.197860,-0.375852,-0.125966,6.0
1,0.349531,0.020488,-0.505710,-0.462156,-0.476915,-0.006576,0.364920,-0.222701,-0.164957,-0.424240,...,0.036171,-0.076903,-0.104565,-0.135202,0.133758,0.069285,-0.107449,-0.315290,-0.282895,9.0
2,0.576189,0.002281,-1.256993,-0.775919,-0.489058,0.593333,0.055468,-0.510053,-0.632301,0.069436,...,-0.541868,-0.218030,-0.128817,0.551386,-0.178715,-0.248486,-0.287657,0.017741,-0.697421,9.0
3,0.658060,-0.037739,-0.624752,-0.798831,-0.798896,1.564715,-0.837681,-0.674321,-0.813789,0.799157,...,-0.682274,-0.238777,0.649445,-0.007221,1.056059,1.311593,-0.189407,-0.478287,-0.239074,4.0
4,0.119220,0.067307,-0.553441,0.016517,-0.488561,0.063167,0.094315,-0.862182,-0.141679,-0.118472,...,-0.182099,-0.262609,-0.270467,-0.447706,0.200527,-0.451478,-0.170289,-0.567564,-0.032854,1.0
5,-0.441965,1.083172,-0.115081,-0.544909,-0.567015,0.046456,-0.255163,-0.172444,-0.500574,-0.363630,...,0.054439,0.313724,0.328341,0.430036,0.276904,-0.019099,0.119374,-0.177221,-0.316098,1.0
6,0.938129,0.137682,-0.591376,0.044833,-0.292282,0.830024,-0.814152,-0.239918,-0.327041,1.964531,...,-0.550348,-0.453861,0.350442,-0.145202,0.374235,-0.054455,-0.082651,-0.458058,-0.039885,2.0
7,0.045929,0.199656,-0.132432,0.019536,0.246675,-0.122472,-0.040682,0.018788,-0.356991,-0.051654,...,-0.330196,-0.230881,-0.290366,-0.433713,-0.145719,-0.432235,-0.226300,0.054540,0.239268,7.0
8,-0.681524,0.145953,0.746743,1.272198,0.043373,0.637533,0.194215,-0.520889,-0.440274,-0.561877,...,-0.590198,-0.557013,-0.634988,-0.337808,0.372047,-0.104201,-0.469096,0.028675,-0.397935,8.0
9,-0.249230,0.164227,-0.955998,-0.712792,-0.535880,-0.256591,-0.393998,-0.669777,-0.625281,-0.253247,...,-0.213020,-0.475241,-0.317314,-0.476635,0.131116,1.309791,-0.473143,-0.188044,-0.250676,3.0


In [37]:
dff = pd.read_csv("/Users/maximemoreau/Documents/ML/ML_deployement/MLproj/data/prod_data.csv", columns = None)

TypeError: read_csv() got an unexpected keyword argument 'columns'

In [40]:
import pandas as pd


# Charger les données
ref_data = df
prod_data = dff

# Réorganiser les colonnes de prod_data pour correspondre à ref_data
prod_data = prod_data.reindex(columns=ref_data.columns, fill_value=None)

# Sauvegarder le fichier corrigé
prod_data.to_csv("/Users/maximemoreau/Documents/ML/ML_deployement/MLproj/data/prod_data.csv", index=False)
print("Colonnes de prod_data.csv alignées avec ref_data.csv")

Colonnes de prod_data.csv alignées avec ref_data.csv


In [42]:
dff

,2024-11-28T03:11:50.246146,3,cat
